In [1]:
import PyPDF2,nltk
import easytextract
import sys

from docx import Document
from docx.shared import Inches

#PDF MIner
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
import pandas as pd
import re

from io import StringIO, BytesIO
from bs4 import BeautifulSoup
import requests

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

C:\Users\AyankoyaKayode\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# Get table of content using PDFMiner.
fp = open('protocols/Immu09.pdf', 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)
#Get the outlines of the document.
outlines = document.get_outlines()

#Get Entire protocol as string
full_protocol= PyPDF2.PdfFileReader(fp)

In [ ]:
df = pd.DataFrame(list(outlines))

In [ ]:
df.columns = ["chapter", "title", "obj", "se", "a"]

#Get index and full title of inclusion criteria
inc_idx = df.index[df["title"].str.contains("Inclusion", na=False)]
inc_title = df[df["title"].str.contains("Inclusion", na=False)]['title'].to_string(index=False)

#Get index and title of section that follows inclusion criteria
inc_idx_end = inc_idx + 1
inc_title_end = df.iloc[inc_idx_end, :]['title'].to_string(index=False)

#Get index and title of section before inclusion criteria
inc_idx_pre = inc_idx -1
inc_title_pre = df.iloc[inc_idx_pre, :]['title'].to_string(index=False)

In [ ]:
inc_title

In [ ]:
inc_title_end

In [ ]:
inc_title_pre

In [ ]:
#Function to convert PDF to Text
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
    
def convert_pdf_to_html(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0 #is for all
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

#Split Document by headings
def iter_headings(paragraphs):
    for paragraph in paragraphs:
        if paragraph.style.name.startswith('Heading'):
            yield paragraph

#for heading in iter_headings(document.paragraphs):
#    print (heading.text)

In [ ]:
txtprotocol = convert_pdf_to_txt('protocols/Immu09.docx')
#txtprotocol = bytes(txtprotocol, 'utf-8').decode('utf-8', 'ignore')

In [ ]:
htmlprotocol = convert_pdf_to_html('protocols/Immu09.pdf')
soup = BeautifulSoup(htmlprotocol, 'html.parser')

In [ ]:
#htm = soup.prettify()
#htm

In [ ]:
with open("protocols/outputx.txt", "w") as file:
    file.write(str(htm.encode('utf-8')))

In [ ]:
with open("protocols/output.txt", "w") as file:
    file.write(str(txtprotocol.encode('utf-8')))

In [42]:
document = Document('protocols/Immu09.docx')


In [92]:
inc = 'Inclusion Criteria'
inc_end = 'Exclusion Criteria'
i = -1
st = 0
en=0
for para in document.paragraphs:
    i += 1
    if para.text == inc:
        st = i     
    elif para.text == inc_end:
        en = i
    
for para in document.paragraphs[st:en]:
    print(para.text)   
    

Inclusion Criteria
Female or male subjects aged ≥18 years at the time of signing the informed consent form (ICF).
Documented evidence of hormone receptor-positive HER2-negative (HR+/HER2-) MBC confirmed	(local  laboratory)  with  the  most recently available	or  newly  obtained tumor


biopsy(within last 12 months) from a locally recurrent or metastatic site(s) and defined per ASCO/CAP criteria as:
HR positive (a tumor is considered HR positive if at least 1% of the cells examined have estrogen and/or progesterone receptors).
Human	epidermal	growth	factor	receptor	2	(HER2)	-negative	(defined	as immunohistochemistry [IHC] ≤2+ or fluorescence in situ hybridization negative).
Availability of archival tumor tissue FFPE) block (within 12 months prior to randomization) or newly acquired biopsy (FFPE block) from a metastatic site.
Refractory to or relapsed after at least 2, and no more than 4, prior systemic chemotherapy regimens for MBC. Adjuvant or neoadjuvant therapy for early stage diseas

In [67]:
a = list(document.paragraphs)

print(a[50])

In [ ]:
allprot = re.split('\n\n',txtprotocol)

In [ ]:
t = ["a", "n", "d"]

for i in t:
    print (str(i))

In [ ]:
crit1 = []

inc = 'Inclusion Criteria'
inc_end = 'Exclusion Criteria'
for i in allprot:
    if i == inc:
        st = allprot.index(i)
        print(st)
    elif i == inc_end:
        en = allprot.index(i)
    if allprot.index(i) >= st or allprot.index(i) < en:
        if 'Exclusion Criteria\n' in str(i):
            break
        crit1.append(i)
     

crit1

In [ ]:
a

In [ ]:

inc_title = "\n\nInclusion Criteria\n\n"
inc_title_end = "\n\nExclusion Criteria\n\n"

In [ ]:
Inc_ = find_between(allprot.upper(),inc_title.replace('\\t', ' ').upper(),inc_title_end.replace('\\t', ' ').upper())

In [ ]:
Inc_

In [ ]:
inc_title.replace('\\t', ' ').upper()

In [ ]:
text_file = open('protocols/as0007.txt', 'w',  encoding='utf8')
text_file.write(allprotocol.upper())
text_file.close()

In [ ]:
file = open('protocols/Immu09.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(file)
aa = ''
for i in range(pdfReader.numPages):
    #print(i)
    pageObj=pdfReader.getPage(i)
    text = pageObj.extractText()
    aa = aa + text 



PDFMIner - https://www.binpress.com/manipulate-pdf-python/

BeautifulSoup - https://www.dataquest.io/blog/web-scraping-tutorial-python/